In [1]:
from pykeen.datasets import WN18RR, FB15k237
import pandas as pd
import numpy as np
import torch
from pykeen.sampling.basic_negative_sampler import BasicNegativeSampler
from transformers import RobertaTokenizer, RobertaForSequenceClassification,Trainer, TrainingArguments
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from functions import *
import os

2023-07-31 18:59:01.500244: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-31 18:59:01.973414: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-31 18:59:04.010827: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### 0 Load dataset

In [2]:
dataset = WN18RR()
path_ent = "data/wn18rr/wn18rr_entity2text.txt"
path_rel = "data/wn18rr/wn18rr_relation2text.txt"
device = "cuda"

### 1 Process dataset

#### 1.1 Load triple labels and ids 

In [6]:
train_label, test_label, val_label =  triple_labels(WN18RR())
train_ids, test_ids, val_ids =  triple_ids(WN18RR())

You're trying to map triples with 212 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3134 triples were filtered out
Reconstructing all label-based triples. This is expensive and rarely needed.
Reconstructing all label-based triples. This is expensive and rarely needed.
You're trying to map triples with 211 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3034 triples were filtered out
Reconstructing all label-based triples. This is expensive and rarely needed.
You're trying to map triples with 212 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3134 triples were filtered out
You're trying to map triples with 211 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3034 triples were fi

#### 1.2 Generate negative samples for each set  

In [7]:
neg_train_triple_labels = neg_sampling(BasicNegativeSampler, train_ids, dataset.training)
neg_test_triple_labels = neg_sampling(BasicNegativeSampler, test_ids, dataset.testing)
neg_val_triple_labels = neg_sampling(BasicNegativeSampler, val_ids, dataset.validation)

You're trying to map triples with 212 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3134 triples were filtered out
You're trying to map triples with 211 entities and 0 relations that are not in the training set. These triples will be excluded from the mapping.
In total 210 from 3034 triples were filtered out


#### 1.3 Load entites and relations of the dataset from exernal file

In [9]:
df_ent, df_rel = load_ent_rel_def("wn18rr", path_ent, path_rel)

#### 1.4 Turn triple labels into KG-NLM suitable input sequence
- For each triple (head, relation, tail) the input sequence should be a string: "head relation tai" 

In [10]:
train_triple_def_pos = triple_def(df_ent, df_rel, train_label, "pos")
train_triple_def_neg = triple_def(df_ent, df_rel, neg_train_triple_labels , "neg")

In [11]:
test_triple_def_pos = triple_def(df_ent, df_rel, test_label , "pos")
test_triple_def_neg = triple_def(df_ent, df_rel, neg_test_triple_labels , "neg")

In [12]:
val_triple_def_pos = triple_def(df_ent, df_rel, val_label , "pos")
val_triple_def_neg = triple_def(df_ent, df_rel, neg_val_triple_labels , "neg")

#### 1.5 Combine positive and negative input sequences for each train/test/val set

In [13]:
train_triples = merge_pos_neg_triples(train_triple_def_pos, train_triple_def_neg)
test_triples = merge_pos_neg_triples(test_triple_def_pos, test_triple_def_neg)
val_triples = merge_pos_neg_triples(val_triple_def_pos, val_triple_def_neg)

#### 1.6 Save triples into separate train/test/val files 

In [ ]:
save_dict(test_triples,'enter_file_name')